# StackOverflow Dataset Exploration

# Use Case : Create cross platform static dataset for query Analysis

## Requirements  

- Use stack overflow dataset from [https://snap.stanford.edu/data/sx-stackoverflow.html](https://snap.stanford.edu/data/sx-stackoverflow.html)
- Convert the data to parquet format
- Create a external table with Hive meta store
- Demonstrate sample queries in SparkSQL, ThriftServer and Hive


------------------------------------------------------------------------------------------------------------------------

## Implementation 
- Explore teh data @ [https://archive.org/details/stackexchange](https://archive.org/details/stackexchange)
- Download the data from [https://archive.org/download/stackexchange](https://archive.org/download/stackexchange)



In [14]:
# !mkdir -p  ~/ssp/data/downloads/stackoverflow/datascience/meta
# !mkdir -p  ~/ssp/data/downloads/stackoverflow/datascience/data
# !axel -n 4 https://archive.org/download/stackexchange/datascience.meta.stackexchange.com.7z -o ~/ssp/data/downloads/stackoverflow/datascience/meta/datascience.meta.stackexchange.com.7z
# !axel -n 4 https://archive.org/download/stackexchange/datascience.stackexchange.com.7z -o ~/ssp/data/downloads/stackoverflow/datascience/data/datascience.stackexchange.com.7z
# !wget https://ia600107.us.archive.org/27/items/stackexchange/readme.txt -O ~/ssp/data/downloads/stackoverflow/datascience/readme.txt

--2020-04-03 19:55:39--  https://ia600107.us.archive.org/27/items/stackexchange/readme.txt
Resolving ia600107.us.archive.org (ia600107.us.archive.org)... 207.241.227.247
Connecting to ia600107.us.archive.org (ia600107.us.archive.org)|207.241.227.247|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4683 (4.6K) [text/plain]
Saving to: ‘/home/mageswarand/ssp/data/downloads/stackoverflow/datascience/readme.txt’

/home/mageswarand/s 100%[===================>]   4.57K  --.-KB/s    in 0s      

2020-04-03 19:55:40 (257 MB/s) - ‘/home/mageswarand/ssp/data/downloads/stackoverflow/datascience/readme.txt’ saved [4683/4683]



In [16]:
!ls ~/ssp/data/downloads/stackoverflow/datascience/

data  meta  readme.txt


In [3]:
!cd ~/ssp/data/downloads/stackoverflow/datascience/meta && 7z x datascience.meta.stackexchange.com.7z



7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_IN,Utf16=on,HugeFiles=on,64 bits,12 CPUs Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz (906EA),ASM,AES-NI)

Scanning the drive for archives:
  0M Sca        1 file, 672641 bytes (657 KiB)

Extracting archive: datascience.meta.stackexchange.com.7z
--
Path = datascience.meta.stackexchange.com.7z
Type = 7z
Physical Size = 672641
Headers Size = 293
Method = BZip2
Solid = +
Blocks = 1

    Everything is Ok

Files: 8
Size:       3964741
Compressed: 672641


In [4]:
!cd ~/ssp/data/downloads/stackoverflow/datascience/data && 7z x datascience.stackexchange.com.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_IN,Utf16=on,HugeFiles=on,64 bits,12 CPUs Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz (906EA),ASM,AES-NI)

Scanning the drive for archives:
  0M Sca        1 file, 48515842 bytes (47 MiB)

Extracting archive: datascience.stackexchange.com.7z
--
Path = datascience.stackexchange.com.7z
Type = 7z
Physical Size = 48515842
Headers Size = 326
Method = BZip2
Solid = +
Blocks = 4

      6% 1 - Comments.xm                     11% 2 - PostHistory.x                       16% 2 - PostHistory.x                       21% 2 - PostHistory.x                       26% 2 - PostHistory.x                       32% 2 - PostHistory.x                       37% 2 - PostHistory.x                       43% 2 - PostHistory.x                       49% 2 - PostHistory.x                       55% 4 - Posts.x                 59% 4 - Posts.x                 64% 4 - Posts.x                 69% 4 - Posts.x                 75%

In [15]:
!cat ~/ssp/data/downloads/stackoverflow/datascience/readme.txt

﻿ - Format: 7zipped
 - Files:
   - **badges**.xml
       - UserId, e.g.: "420"
       - Name, e.g.: "Teacher"
       - Date, e.g.: "2008-09-15T08:55:03.923"
   - **comments**.xml
       - Id
       - PostId
       - Score
       - Text, e.g.: "@Stu Thompson: Seems possible to me - why not try it?"
       - CreationDate, e.g.:"2008-09-06T08:07:10.730"
       - UserId
   - **posts**.xml
       - Id
       - PostTypeId
          - 1: Question
          - 2: Answer
       - ParentID (only present if PostTypeId is 2)
       - AcceptedAnswerId (only present if PostTypeId is 1)
       - CreationDate
       - Score
       - ViewCount
       - Body
       - OwnerUserId
       - LastEditorUserId
       - LastEditorDisplayName="Jeff Atwood"
       - LastEditDate="2009-03-05T22:28:34.823"
       - LastActivityDate="2009-03-11T12:51:01.480"
       - CommunityOwnedDate="2009-03-11T12:51:01.480"
       - ClosedDate="2009-03-11T12:51:01.480"
       - Title=
       - Tags=
       - AnswerCount
       -

In [16]:
!ls ~/ssp/data/downloads/stackoverflow/datascience/data

Badges.xml			  PostHistory.xml  Tags.xml
Comments.xml			  PostLinks.xml    Users.xml
datascience.stackexchange.com.7z  Posts.xml	   Votes.xml


In [53]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnull, desc
import os

In [3]:
!ls ../../libs/spark-xml_2.12-0.5.0.jar

ls: cannot access '../../libs/spark-xml_2.12-0.5.0.jar': No such file or directory


In [4]:
spark_master = "spark://IMCHLT276:7077"
spark = SparkSession.builder. \
                appName("twitter_stream"). \
                config("spark.jars", "../../libs/spark-xml_2.11-0.9.0.jar"). \
                config("spark.executor.memory", "3g"). \
                config("spark.executor.cores", 3). \
                config("spark.cores.max", 6). \
                master(spark_master). \
                enableHiveSupport(). \
                getOrCreate()

In [5]:
spark

In [17]:
!ls ~/ssp/data/downloads/stackoverflow/datascience/data

Badges.xml			  PostHistory.xml  Tags.xml
Comments.xml			  PostLinks.xml    Users.xml
datascience.stackexchange.com.7z  Posts.xml	   Votes.xml


In [37]:
def load_data(path, debug=True):
    """
    Loads all the Stackoverflow files as PySpark DataFrame. Column names are prefixed with `_` by default.
    :path : Root directory of the zip extracted folder
    returns: Dictionary of DataFrames where each key(lowercase) represents the respective XML file
    """
    files = ["Badges.xml", "PostHistory.xml", "Tags.xml", "Comments.xml", "PostLinks.xml", "Users.xml", "Posts.xml", "Votes.xml"]
    dfs = {}
    for key in files:
        full_path = path + key
        key_ = key.lower().replace(".xml", "")
        dfs[key_] = spark.read.format("com.databricks.spark.xml").option("rootTag", "Tags").option("attributePrefix", "_").option("rowTag", "row").load(full_path)
        if debug:
            print(f"File : {key}")
            dfs[key_].show(5)
    return dfs

path = "file:///" + os.path.expanduser("~/ssp/data/downloads/stackoverflow/datascience/data/")
dfs = load_data(path=path) 

File : Badges.xml
+------+--------------------+---+--------------+---------+-------+
|_Class|               _Date|_Id|         _Name|_TagBased|_UserId|
+------+--------------------+---+--------------+---------+-------+
|     3|2014-05-13T23:06:...|  1|      Informed|    false|      1|
|     3|2014-05-13T23:11:...|  2|Autobiographer|    false|      2|
|     3|2014-05-13T23:20:...|  3|Autobiographer|    false|      4|
|     3|2014-05-13T23:20:...|  4|Autobiographer|    false|      5|
|     3|2014-05-13T23:20:...|  5|Autobiographer|    false|      8|
+------+--------------------+---+--------------+---------+-------+
only showing top 5 rows

File : PostHistory.xml
+--------+--------------------+---+------------------+-------+--------------------+--------------------+----------------+-------+
|_Comment|       _CreationDate|_Id|_PostHistoryTypeId|_PostId|       _RevisionGUID|               _Text|_UserDisplayName|_UserId|
+--------+--------------------+---+------------------+-------+---------

**Find top 5 Posts with more answers and find what those questions are?**


In [60]:
questions = dfs["posts"].where(~isnull(col("_AnswerCount"))).sort(desc("_AnswerCount")).select(["_AnswerCount", "_Body"]).collect()[:5]

In [62]:
for q in questions:
    print(q["_Body"])
    print("-------------------------------------------")

<p>One of the common problems in data science is gathering data from various sources in a somehow cleaned (semi-structured) format and combining metrics from various sources for making a higher level analysis. Looking at the other people's effort, especially other questions on this site, it appears that many people in this field are doing somewhat repetitive work. For example analyzing tweets, facebook posts, Wikipedia articles etc. is a part of a lot of big data problems.</p>

<p>Some of these data sets are accessible using public APIs provided by the provider site, but usually, some valuable information or metrics are missing from these APIs and everyone has to do the same analyses again and again. For example, although clustering users may depend on different use cases and selection of features, but having a base clustering of Twitter/Facebook users can be useful in many Big Data applications, which is neither provided by the API nor available publicly in independent data sets.</p>


# Stay tuned for more updates!

https://towardsdatascience.com/getting-started-with-data-analytics-using-jupyter-notebooks-pyspark-and-docker-57c1aaab2408

In [73]:
from pyspark.sql.types import StructType, StringType, IntegerType, ArrayType


In [74]:
def get_schema():
    # define the schema to extract the data we are interested
    # https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object
    urls = ArrayType(StructType(). \
                     add("expanded_url", StringType(), True))

    media = ArrayType(StructType(). \
                      add("media_url", StringType(), True). \
                      add("media_url_https", StringType(), True))

    # Tweet -> Entities{} -> Urls[] -> Media[]
    entities = StructType(). \
        add("urls", urls, True). \
        add("media", media)

    schema = StructType(). \
        add('id_str', StringType(), False). \
        add('created_at', StringType(), False). \
        add('source', StringType(), False). \
        add('text', StringType(), False). \
        add('extended_tweet', StructType().add("full_text", StringType(), True), True). \
        add('entities', entities, False). \
        add('retweeted_status', StructType().add('extended_tweet', StructType().add("full_text", StringType(), True), True).add('user', StructType().add('description', StringType())), True). \
        add('geo', StringType(), True). \
        add('retweet_count', IntegerType(), True)

    return schema

In [82]:
test_files_path = "file:///opt/vlab/gyan42/spark-streaming-playground/data/streams/tweets/"
sdf = spark.readStream.format("json").schema(get_schema()).load(test_files_path).repartition(5)

In [91]:
count_acc = spark.sparkContext.accumulator(0)

In [100]:
def foreach_batch_function(df, epoch_id):
    # Transform and write batchDF
    global count_acc
    print("=================================================================================\n\n")
    df.show(2, True)
    
    count = df.count()
    count_acc += count

sdf.writeStream.foreachBatch(foreach_batch_function).start().processAllAvailable()
print(count_acc.value)



+-------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+----+-------------+
|             id_str|          created_at|              source|                text|extended_tweet|            entities|    retweeted_status| geo|retweet_count|
+-------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+----+-------------+
|1246751216496816129|Sun Apr 05 10:47:...|<a href="http://t...|RT @smolkjd: afte...|          null|[[], [[http://pbs...|[, ['The standard...|null|            0|
|1246751191511584769|Sun Apr 05 10:47:...|<a href="http://t...|RT @morganhousel:...|          null|               [[],]|[, [@collabfund

...|null|            0|
+-------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+----+-------------+
only showing top 2 rows

6001


In [90]:
res
#

False

https://lgoerl.github.io/spark-overflow-analysis.html